### mounting inputcontainer  for accessing source files 

In [ ]:

dbutils.fs.mount(
    source = "wasbs://sourcecont@ottstg1.blob.core.windows.net",
    mount_point = '/mnt/sourcedata',
    extra_configs ={'fs.azure.account.key.ottstg1.blob.core.windows.net':'vC2Q8bwxujkAp050POerQk7aTcnocahDATSflkdOB4BsO+yCfJLjOZ3IWJDP0QvDKZ7KnaWaXjTd+ASt5Po0yQ=='}
)
df = spark.read.format("csv").options(inferSchema=True,header=True).load("/mnt/sourcedata/*")
df.show(3)

+---------+--------------------+--------------------+------+-----+-----+-----------+-----------------+-------+--------------------+-------------+--------------------+--------------------+----------------+---------------+--------------------+---------+-----------+----------+
|  imdb_id|               title|                plot|  type|rated| year|released_at|         added_at|runtime|               genre|     director|              writer|              actors|        language|        country|              awards|metascore|imdb_rating|imdb_votes|
+---------+--------------------+--------------------+------+-----+-----+-----------+-----------------+-------+--------------------+-------------+--------------------+--------------------+----------------+---------------+--------------------+---------+-----------+----------+
|tt0147800|10 Things I Hate ...|A pretty, popular...| movie|PG-13| 1999|31 Mar 1999|November 12, 2019| 97 min|Comedy, Drama, Ro...|   Gil Junger|Karen McCullah, K...|Heath Led

### Importing required libraries

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

### selecting required columns and Preprocessing year,imdb_votes,release_date column

In [ ]:
df1=df.select(col('year'),col('director'),col('imdb_votes'),col('title'),col('awards'),col('released_at'))

df2 = df1.withColumn('newYr', when(length(df1.year) >= 4 , df1.year[0:4]).otherwise(0)).withColumn('yr',when(col('newYr').cast("int").isNotNull(), col('newYr')).otherwise(0)).drop('year','newYr')

df3 = df2.select(col('title'),col('yr').cast('int'),col('director'),col('awards'),col('released_at'), regexp_replace(col('imdb_votes'),',','').cast('int').alias('votes'))

df3=df3.withColumn('release_date',split(df1.released_at,' ')[2]).drop('released_at').withColumn('releaseYr',when(col('release_date').cast("int").isNotNull(), col('release_date')).otherwise(0)).drop('release_date')

df5=df3.select(col('title'),col('yr'),col('director'),col('awards'),col('votes'),col('releaseYr').cast('int').alias('release_year')).filter((col('director')!='N/A') & (col('yr')!=0) & (col('title')!='N/A'))
df5.show(2)

+--------------------+----+-------------+--------------------+------+------------+
|               title|  yr|     director|              awards| votes|release_year|
+--------------------+----+-------------+--------------------+------+------------+
|10 Things I Hate ...|1999|   Gil Junger|2 wins & 13 nomin...|283945|        1999|
|      101 Dalmatians|1996|Stephen Herek|Nominated for 1 G...| 97785|        1996|
+--------------------+----+-------------+--------------------+------+------------+
only showing top 2 rows



### extracting total wins from awards

In [ ]:
df6 = df5.withColumn(
    'win',
    when(regexp_extract(col('awards'), r'(\d+) win', 1) != '', regexp_extract(col('awards'), r'(\d+) win', 1))
        .otherwise(0)
        .cast('int')) \
.withColumn(
    'won',
    when(regexp_extract('awards', r'Won (\d+)', 1) != '', regexp_extract('awards', r'Won (\d+)', 1))
        .otherwise(0)
        .cast('int')) \
.withColumn("Total_win",col('win')+col('won')) \
.drop("win","won")
df6.show(2)

+--------------------+----+-------------+--------------------+------+------------+---------+
|               title|  yr|     director|              awards| votes|release_year|Total_win|
+--------------------+----+-------------+--------------------+------+------------+---------+
|10 Things I Hate ...|1999|   Gil Junger|2 wins & 13 nomin...|283945|        1999|        2|
|      101 Dalmatians|1996|Stephen Herek|Nominated for 1 G...| 97785|        1996|        3|
+--------------------+----+-------------+--------------------+------+------------+---------+
only showing top 2 rows



### extrating nominations from awards

In [ ]:
df7 = df6.withColumn(
    'nomination',
    when(regexp_extract(col('awards'), r'(\d+) nomin', 1) != '', regexp_extract(col('awards'), r'(\d+) nomin', 1))
    .otherwise(0)
    .cast('int')) \
.withColumn('nominated',
     when(regexp_extract('awards', r'Nominated for (\d+)', 1) != '', regexp_extract('awards', r'Nominated for (\d+)', 1))
     .otherwise(0)
     .cast('int')) \
     .withColumn("Total_Nominated",col('nomination')+col('nominated')) \
     .drop("nomination","nominated")

df7.show(2)

+--------------------+----+-------------+--------------------+------+------------+---------+---------------+
|               title|  yr|     director|              awards| votes|release_year|Total_win|Total_Nominated|
+--------------------+----+-------------+--------------------+------+------------+---------+---------------+
|10 Things I Hate ...|1999|   Gil Junger|2 wins & 13 nomin...|283945|        1999|        2|             13|
|      101 Dalmatians|1996|Stephen Herek|Nominated for 1 G...| 97785|        1996|        3|             10|
+--------------------+----+-------------+--------------------+------+------------+---------+---------------+
only showing top 2 rows



### 1. Which director's movies has the highest votes? and for that director, which year he has the highest votes?

In [ ]:
df7.sort(col('votes').desc()).show(1)

+------------+----+-----------+--------------------+-------+------------+---------+---------------+
|       title|  yr|   director|              awards|  votes|release_year|Total_win|Total_Nominated|
+------------+----+-----------+--------------------+-------+------------+---------+---------------+
|The Avengers|2012|Joss Whedon|Nominated for 1 O...|1225316|        2012|       38|             80|
+------------+----+-----------+--------------------+-------+------------+---------+---------------+
only showing top 1 row



### 2. Total number of movies released in 1999, by director wise.

In [ ]:
df5.filter((df5.release_year==1999) & (df5.director!="N/A")).groupBy('director').agg(count('title').alias('movies_count_in_1999')).sort(col('movies_count_in_1999').desc()).show(50)

+--------------------+--------------------+
|            director|movies_count_in_1999|
+--------------------+--------------------+
|     Kenneth Johnson|                   2|
|          Gil Junger|                   1|
|       Maurice Joyce|                   1|
|       David Kellogg|                   1|
|        LeVar Burton|                   1|
|Robert C. Ramirez...|                   1|
|          Rod Daniel|                   1|
|         David Lynch|                   1|
|         Steve Boyum|                   1|
|         Greg Beeman|                   1|
|      Duwayne Dunham|                   1|
|John Lasseter, As...|                   1|
|        Rob Marshall|                   1|
|      Paul Schneider|                   1|
|        George Lucas|                   1|
|Jun Falkenstein, ...|                   1|
|       Donald Petrie|                   1|
+--------------------+--------------------+



### 3. Which year movies got highest awards? And it belongs to which director?

In [ ]:
df7.filter(col('director')!='N/A').sort(col("Total_win").desc()).select("title","yr","director","Total_win").show(1)

+-------------+----+------------+---------+
|        title|  yr|    director|Total_win|
+-------------+----+------------+---------+
|Black Panther|2018|Ryan Coogler|      111|
+-------------+----+------------+---------+
only showing top 1 row



### 4.Total number of movies nominated for awards between 1998-2018?

In [ ]:
df7.filter((df7.yr>=1998) & (df7.yr<=2018) & (df7.Total_Nominated>0)).count()

Out[18]: 260

### connecting to Azure SQL DB

In [ ]:
server_name = "disneydbserver.database.windows.net"
database_name = "ottsqldb"
url = "jdbc:sqlserver://{0};database={1}".format(server_name, database_name)
username = "ottsqladminuser"
password = "Srkp@594"

### Writing the processed DataFrame and Analysis results to Azure SQL DB

In [ ]:
processeddf=df7.sort(col('votes').desc())
director_movies=df5.filter((df5.release_year==1999)).groupBy('director').agg(count('title').alias('movies_count_in_1999')).sort(col('movies_count_in_1999').desc())
movies_awardsdf=df7.sort(col("Total_win").desc()).select("title","yr","director","Total_win")

processeddf.write.format("jdbc").option("url", url).option("dbtable", "disneymovies").option("user", username).option("password", password).mode('append').save()
director_movies.write.format("jdbc").option("url", url).option("dbtable", "director_movies").option("user", username).option("password", password).mode('append').save()
movies_awardsdf.write.format("jdbc").option("url", url).option("dbtable", "movies_awards").option("user", username).option("password", password).mode('append').save()


### unmounting the path

In [ ]:
dbutils.fs.unmount("/mnt/sourcedata")

/mnt/sourcedata has been unmounted.
Out[22]: True